# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_norway/model/')

In [2]:
import pandas as pd
import numpy as np
import pickle
import itertools
import random
import torch
import optuna
from model.utils import *

/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-18 13:44:08.518742: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from pipeline_config import *

# Seeds for Comparisons:

In [4]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

# Read Data

In [5]:
with open(son_parent_path, 'rb') as f:
    son_parent_dict = pickle.load(f)

with open(train_dataset_dict_path, 'rb') as f:
    train_dataset_dict = pickle.load(f)

with open(test_dataset_dict_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [6]:
len(list(test_dataset_dict.keys()))

52

# Hierarchical GRU

In [7]:
def objective(trial):
    with open(son_parent_path, 'rb') as f:
        son_parent_dict = pickle.load(f)

    with open(train_dataset_dict_path, 'rb') as f:
        train_dataset_dict = pickle.load(f)

    with open(test_dataset_dict_path, 'rb') as f:
        test_dataset_dict = pickle.load(f)

    with open(category_id_to_category_name_path, 'rb') as f:
        category_id_to_name_dict = pickle.load(f)
        
    with open(categories_per_indent_path, 'rb') as f:
        categories_per_indent_dict = pickle.load(f)

    weights_path = weightspath
    
    #--------------------------------------------------------------------------------------------------------------------------------------#

    loss_coef = trial.suggest_float('loss_coef_1',  1e-10, 1e-1, log=True)
    Lr = trial.suggest_float('Lr', 1e-5, 1e-1, log=True)

    hgru_models = {}
    num_categories = 0

    for indent in sorted(list(categories_per_indent_dict.keys())):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]:
                loss_coef=0
                parent_weights=0
            else:
                son = category
                parent = son_parent_dict[son]
                parent_name = category_id_to_name_dict[parent]
                parent_model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
                parent_optimizer = torch.optim.AdamW(parent_model.parameters(), lr=Lr)
                parent_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(weights_path+parent_name+'.pt', parent_model, parent_optimizer)
                parent_weights = unify_model_weights(parent_model)

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            min_error = training_and_evaluation(trial, model, train_dataloader, test_dataloader, optimizer, category_name, parent_weights, loss_coef, path=saving_param_path)
            hgru_models[category] = min_error

    average_error = sum(list(hgru_models.values()))/len(list(hgru_models.values()))
    return average_error


In [8]:
EPOCHS = 30
    
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2023-06-18 13:44:13,315] A new study created in memory with name: no-name-57bf6f83-d0a2-4394-8684-36b16ae39481


num categories: 1
category id|name: 0|All-items
num categories: 2
category id|name: 2|Alcoholic beverages and tobacco


/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudSto

num categories: 3
category id|name: 3|Clothing and footwear
num categories: 4
category id|name: 8|Communications
num categories: 5
category id|name: 10|Education
num categories: 6
category id|name: 1|Food and non-alcoholic beverages
num categories: 7
category id|name: 5|Furnishings, household equipment and routine maintenance
num categories: 8
category id|name: 6|Health
num categories: 9
category id|name: 4|Housing, water, electricity, gas and other fuels
num categories: 10
category id|name: 12|Miscellaneous goods and services
num categories: 11
category id|name: 9|Recreation and culture
num categories: 12
category id|name: 11|Restaurants and hotels
num categories: 13
category id|name: 7|Transport
num categories: 14
category id|name: 45|Accommodation services
num categories: 15
category id|name: 19|Actual rentals for housing
num categories: 16
category id|name: 15|Alcoholic beverages
num categories: 17
category id|name: 38|Audio-visual, photographic and information processing equipment

[I 2023-06-18 13:57:13,404] Trial 0 finished with value: 0.13539435775377429 and parameters: {'loss_coef_1': 0.023922439871367416, 'Lr': 0.0014588196120359218}. Best is trial 0 with value: 0.13539435775377429.


...

[I 2023-06-18 19:51:58,289] Trial 28 finished with value: 0.12334664434624407 and parameters: {'loss_coef_1': 0.009428742519331986, 'Lr': 0.023284660569143665}. Best is trial 22 with value: 0.108527746193551.


num categories: 1
category id|name: 0|All-items
num categories: 2
category id|name: 2|Alcoholic beverages and tobacco


/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudSto

num categories: 3
category id|name: 3|Clothing and footwear
num categories: 4
category id|name: 8|Communications
num categories: 5
category id|name: 10|Education
num categories: 6
category id|name: 1|Food and non-alcoholic beverages
num categories: 7
category id|name: 5|Furnishings, household equipment and routine maintenance
num categories: 8
category id|name: 6|Health
num categories: 9
category id|name: 4|Housing, water, electricity, gas and other fuels
num categories: 10
category id|name: 12|Miscellaneous goods and services
num categories: 11
category id|name: 9|Recreation and culture
num categories: 12
category id|name: 11|Restaurants and hotels
num categories: 13
category id|name: 7|Transport
num categories: 14
category id|name: 45|Accommodation services
num categories: 15
category id|name: 19|Actual rentals for housing
num categories: 16
category id|name: 15|Alcoholic beverages
num categories: 17
category id|name: 38|Audio-visual, photographic and information processing equipment

[I 2023-06-18 20:02:28,584] Trial 29 finished with value: 0.11299892561382278 and parameters: {'loss_coef_1': 0.018578335487509843, 'Lr': 0.04971910141225333}. Best is trial 22 with value: 0.108527746193551.


In [9]:
best_trial = study.best_trial
best_trial

FrozenTrial(number=22, state=1, values=[0.108527746193551], datetime_start=datetime.datetime(2023, 6, 18, 18, 24, 35, 685034), datetime_complete=datetime.datetime(2023, 6, 18, 18, 40, 8, 174442), params={'loss_coef_1': 0.03881843050613352, 'Lr': 0.04329823452489634}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.019637041858264377, 1: 0.0121331587433815, 2: 0.0121331587433815, 3: 0.0121331587433815, 4: 0.0121331587433815, 5: 0.0121331587433815, 6: 0.0121331587433815, 7: 0.01112799665757588, 8: 0.01112799665757588, 9: 0.01112799665757588, 10: 0.010965960366385323, 11: 0.010965960366385323, 12: 0.010965960366385323, 13: 0.010965960366385323, 14: 0.010965960366385323, 15: 0.010965960366385323, 16: 0.010965960366385323, 17: 0.010965960366385323, 18: 0.010965960366385323, 19: 0.010896664644990648, 20: 0.010896664644990648, 21: 0.010896664644990648, 22: 0.010896664644990648, 23: 0.010896664644990648, 24: 0.010896664644990648, 25: 0.010896664644990648, 26: 0.010896664644990648, 27